### Parse elog

In [10]:
import os
from glob import glob

import pandas as pd

from neslter.parsing.files import Resolver

INSTRUMENT = 'Instrument'
ACTION = 'Action'
COMMENT = 'Comment'
STATION = 'Station'
CAST = 'Cast'
DATETIME = 'dateTime8601'
LAT='Latitude'
LON='Longitude'
MESSAGE_ID = 'Message ID'

ELOG_COLUMNS = [MESSAGE_ID, DATETIME, INSTRUMENT, ACTION, STATION, CAST, LAT, LON, COMMENT]

cruise = 'en627'

# this assumes that the resolver is configured for your file locations
elog_dir = Resolver().raw_directory('elog', cruise)

candidates = glob(os.path.join(elog_dir, 'R2R_ELOG_*_FINAL_EVENTLOG*.csv'))
assert len(candidates) == 1, 'cannot find event log at {}'.format(elog_dir)
elog_path = candidates[0]

raw = pd.read_csv(elog_path) # defaults work
raw[DATETIME] = pd.to_datetime(raw[DATETIME], utc=True) # parse date column
raw = raw[ELOG_COLUMNS] # retain only the columns we want to use
raw.head()

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
0,1,2019-02-02 04:24:05+00:00,Ship,startCruise,NaN,NaN,NaN,NaN,NaN
1,2,2019-02-01 16:01:51+00:00,Echosounder12,start,NaN,NaN,NaN,NaN,NaN
2,3,2019-02-01 16:02:20+00:00,UHDAS for both ADCPs,start,NaN,NaN,NaN,NaN,NaN
3,4,2019-02-01 16:02:49+00:00,ADCP300,start,NaN,NaN,NaN,NaN,NaN
4,5,2019-02-01 16:03:26+00:00,ADCP75,start,NaN,NaN,NaN,NaN,NaN


### Replace all CTD events with correct ones derived from CTD metadata product

In [2]:

noctd = raw[~(raw[INSTRUMENT] == 'CTD911')]
noctd.head()

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
0,1,2019-02-02 04:24:05+00:00,Ship,startCruise,NaN,NaN,NaN,NaN,NaN
1,2,2019-02-01 16:01:51+00:00,Echosounder12,start,NaN,NaN,NaN,NaN,NaN
2,3,2019-02-01 16:02:20+00:00,UHDAS for both ADCPs,start,NaN,NaN,NaN,NaN,NaN
3,4,2019-02-01 16:02:49+00:00,ADCP300,start,NaN,NaN,NaN,NaN,NaN
4,5,2019-02-01 16:03:26+00:00,ADCP75,start,NaN,NaN,NaN,NaN,NaN


In [3]:
# put CTD metadata in the right columns
import numpy as np

from neslter.workflow.ctd import CtdMetadataWorkflow

md = CtdMetadataWorkflow(cruise).get_product()
md.head()

,cruise,cast,date,latitude,longitude,nearest_station
0,EN627,1,2019-02-01 21:05:30+00:00,41.197667,-70.882833,L1
1,EN627,3,2019-02-01 21:39:15+00:00,41.197000,-70.881833,L1
2,EN627,4,2019-02-01 21:53:45+00:00,41.198333,-70.879833,L1
3,EN627,5,2019-02-02 02:18:32+00:00,41.028667,-70.763500,u2a
4,EN627,6,2019-02-02 04:03:51+00:00,41.025500,-70.990333,d2a


In [4]:
# reorganize metadata
md.pop('cruise')
md.insert(0, MESSAGE_ID, -9999) # this will later be discarded
md.insert(1, DATETIME, pd.to_datetime(md.pop('date')))
md.insert(2, INSTRUMENT, 'CTD911')
md.insert(3, ACTION, 'deploy')
md.insert(4, STATION, md.pop('nearest_station'))
md.insert(5, CAST, md.pop('cast'))
md.insert(6, LAT, md.pop('latitude'))
md.insert(7, LON, md.pop('longitude'))
md.insert(8, COMMENT, np.nan)

# add ctd events to the elog

wctd = pd.concat([noctd, md], sort=False)
wctd.tail()

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
39,-9999,2019-02-05 21:16:18+00:00,CTD911,deploy,L4,41,40.695167,-70.878167,NaN
40,-9999,2019-02-05 22:51:18+00:00,CTD911,deploy,L3,42,40.861833,-70.883833,NaN
41,-9999,2019-02-06 00:21:45+00:00,CTD911,deploy,L2,43,41.028333,-70.884000,NaN
42,-9999,2019-02-06 01:42:34+00:00,CTD911,deploy,L1,44,41.195000,-70.883000,NaN
43,-9999,2019-02-06 04:23:26+00:00,CTD911,deploy,MVCO,45,41.320167,-70.572333,NaN


### Add events specified in the additions file

In [5]:
addns_file = os.path.join(elog_dir, 'R2R_ELOG_{}_additions.xlsx'.format(cruise))
assert os.path.exists(addns_file)
addns = pd.read_excel(addns_file)
addns[DATETIME] = pd.to_datetime(addns[DATETIME], utc=True)
addns.insert(0, MESSAGE_ID, -9999)
addns.insert(5, CAST, np.nan)
addns.insert(6, LAT, np.nan)
addns.insert(7, LON, np.nan)
addns

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
0,-9999,2019-02-02 16:40:34+00:00,UHDAS for both ADCPs,stop,NaN,NaN,NaN,NaN,NaN
1,-9999,2019-02-02 16:44:00+00:00,UHDAS for both ADCPs,start,NaN,NaN,NaN,NaN,NaN


In [6]:
wadds = pd.concat([wctd, addns], sort=False)
wadds.tail()

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
41,-9999,2019-02-06 00:21:45+00:00,CTD911,deploy,L2,43,41.028333,-70.884000,NaN
42,-9999,2019-02-06 01:42:34+00:00,CTD911,deploy,L1,44,41.195000,-70.883000,NaN
43,-9999,2019-02-06 04:23:26+00:00,CTD911,deploy,MVCO,45,41.320167,-70.572333,NaN
0,-9999,2019-02-02 16:40:34+00:00,UHDAS for both ADCPs,stop,NaN,NaN,NaN,NaN,NaN
1,-9999,2019-02-02 16:44:00+00:00,UHDAS for both ADCPs,start,NaN,NaN,NaN,NaN,NaN


### Apply time corrections from the corrections file

In [7]:
corr_path = os.path.join(elog_dir, 'R2R_ELOG_{}_corrections.xlsx'.format(cruise))
assert os.path.exists(corr_path)

corr = pd.read_excel(corr_path)
corr[DATETIME] = pd.to_datetime(corr[DATETIME], utc=True)
corr.pop('Instrument')
corr.pop('Action')
merged = wadds.merge(corr, on=MESSAGE_ID, how='left')
DATETIME_X = '{}_x'.format(DATETIME)
DATETIME_Y = '{}_y'.format(DATETIME)
dt = pd.to_datetime(merged[DATETIME_Y].combine_first(merged[DATETIME_X]), utc=True)
merged.pop(DATETIME_X)
merged.pop(DATETIME_Y)
merged.insert(1, DATETIME, dt)
wcorr = merged
wcorr.head()

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
0,1,2019-02-01 16:00:14+00:00,Ship,startCruise,NaN,NaN,NaN,NaN,NaN
1,2,2019-02-01 16:01:51+00:00,Echosounder12,start,NaN,NaN,NaN,NaN,NaN
2,3,2019-02-01 16:02:20+00:00,UHDAS for both ADCPs,start,NaN,NaN,NaN,NaN,NaN
3,4,2019-02-01 16:02:49+00:00,ADCP300,start,NaN,NaN,NaN,NaN,NaN
4,5,2019-02-01 16:03:26+00:00,ADCP75,start,NaN,NaN,NaN,NaN,NaN


### Add underway locations for events where lat/lon is not provided

In [8]:
from neslter.workflow.underway import UnderwayWorkflow

uw = UnderwayWorkflow(cruise).get_product()

from neslter.workflow.underway import TimeToLocation

ttl = TimeToLocation(uw)
uw_lat = wcorr[DATETIME].map(lambda t: ttl.time_to_lat(t))
uw_lon = wcorr[DATETIME].map(lambda t: ttl.time_to_lon(t))

df = wcorr
df[LAT] = df[LAT].combine_first(uw_lat)
df[LON] = df[LON].combine_first(uw_lon)

wll = df
wll.head()

,Message ID,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
0,1,2019-02-01 16:00:14+00:00,Ship,startCruise,NaN,NaN,41.5863,-71.4091,NaN
1,2,2019-02-01 16:01:51+00:00,Echosounder12,start,NaN,NaN,41.5861,-71.4088,NaN
2,3,2019-02-01 16:02:20+00:00,UHDAS for both ADCPs,start,NaN,NaN,41.5858,-71.4083,NaN
3,4,2019-02-01 16:02:49+00:00,ADCP300,start,NaN,NaN,41.5858,-71.4083,NaN
4,5,2019-02-01 16:03:26+00:00,ADCP75,start,NaN,NaN,41.5857,-71.4074,NaN


### Remove message ID and produce CSV file

In [9]:
final = wll.copy()
final.pop(MESSAGE_ID)
final = final.sort_values(DATETIME)
final.to_csv('en627_elog.csv', index=None)